In [83]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql.functions import *

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 85, Finished, Available, Finished)

# Cleaning Customer file

In [84]:
df = (
    spark.read.format("csv")
    .option("header", "true")
    .load(
        "abfss://cad6716d-54df-4655-aed7-d883bc9a65b4@onelake.dfs.fabric.microsoft.com/"
        "b90738e8-b697-45a2-a1e2-b96143f7b0f5/Files/Dataset/customers.csv"
    )
)

display(df)


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 86, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 99116457-d593-4ec7-a7f7-a369f2ae2840)

In [85]:
#row count
df.count()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 87, Finished, Available, Finished)

50000

In [86]:
df=df.dropDuplicates()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 88, Finished, Available, Finished)

In [87]:
df.count()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 89, Finished, Available, Finished)

48979

In [88]:
df.groupBy("customer_id").count().filter("count > 1").show(50)


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 90, Finished, Available, Finished)

+--------------------+-----+
|         customer_id|count|
+--------------------+-----+
|bf4e3ff5-40dd-43c...|    2|
|f972fa2e-ddef-448...|    2|
|8504c32c-baf6-42f...|    2|
|e6328572-d3d3-4dd...|    2|
|201135c2-aee7-481...|    2|
|8e05ea34-a9b1-41f...|    2|
|30872013-9450-416...|    2|
|a6b037d3-b051-4a0...|    2|
|a5c6a9df-4fd6-4d8...|    2|
|c9b1f8ae-ef5e-4a3...|    2|
|5dbd2c55-3e4f-4b9...|    2|
|1c37dd6c-9e54-4f4...|    2|
|00682ed4-54ac-434...|    2|
|dfa46fea-38c3-40d...|    2|
|38093645-650a-467...|    2|
|aee0a8c7-af75-4f1...|    2|
|7a239b1c-546b-4e8...|    2|
|c6cea585-8a29-491...|    2|
|a38f0002-1560-4f9...|    2|
|778cf384-044e-41d...|    2|
|0d2319c7-9fcf-49d...|    2|
|98e6b47c-1f48-4c1...|    2|
|af56b5dd-49fc-4d8...|    2|
|3eb52b42-9dbd-49b...|    2|
|f80843aa-7d40-44b...|    2|
|2063d6c7-54a3-4c2...|    2|
|8bd37bdc-e3dd-489...|    2|
|5d3f9577-1d6e-423...|    2|
|d9d672fd-358f-4f1...|    2|
|dd4db7d6-0731-45f...|    2|
|d7077c04-a1fd-4ab...|    2|
|e14eb90d-17ec

In [89]:
df = df.dropDuplicates(["customer_id"])


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 91, Finished, Available, Finished)

In [90]:
df.groupBy("customer_id").count().filter("count > 1").show()


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 92, Finished, Available, Finished)

+-----------+-----+
|customer_id|count|
+-----------+-----+
+-----------+-----+



In [91]:
# df.filter(col("customer_id").isNull()).count()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 93, Finished, Available, Finished)

In [92]:
for c in df.columns:
    null_count= df.filter(col(c).isNull()).count()
    print(c, null_count)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 94, Finished, Available, Finished)

customer_id 0
first_name 0
last_name 0
email 1000
phone_number 0
gender 0
dob 0
signup_date 0
address 0
city 0
state 0
country 0
device_id(s) 0
source 0


In [93]:
df.select("gender").distinct().show()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 95, Finished, Available, Finished)

+------+
|gender|
+------+
|     F|
|     M|
|     O|
+------+



In [94]:
df.select("source").distinct().show()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 96, Finished, Available, Finished)

+--------+
|  source|
+--------+
|referral|
|     app|
|     web|
+--------+



In [95]:
df.printSchema()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 97, Finished, Available, Finished)

root
 |-- customer_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- signup_date: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- device_id(s): string (nullable = true)
 |-- source: string (nullable = true)



In [96]:
from pyspark.sql.functions import col, to_date

df = df.withColumn("dob", to_date(col("dob"), "yyyy-MM-dd")) \
       .withColumn("signup_date", to_date(col("signup_date"), "yyyy-MM-dd"))


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 98, Finished, Available, Finished)

In [97]:
df.printSchema()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 99, Finished, Available, Finished)

root
 |-- customer_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- signup_date: date (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- device_id(s): string (nullable = true)
 |-- source: string (nullable = true)



In [98]:
df=df.withColumn("first_name",lower(col("first_name")))

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 100, Finished, Available, Finished)

In [99]:
df=df.withColumn("last_name",lower(col("last_name")))

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 101, Finished, Available, Finished)

In [100]:
# regexp_replace --> regexp_replace(column, pattern, replacement) -->Find pattern → replace with value
# regexp_replace() = what change to make
# withColumn() = where to apply that change
# [^0-9] not a number

df=df.withColumn("phone_number",regexp_replace(col("phone_number"),"[^0-9]", ""))


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 102, Finished, Available, Finished)

In [101]:
df = (
    df.withColumn(
        "first_name",
        regexp_replace(col("first_name"), "[^a-zA-Z]", "")
    )
    .withColumn(
        "last_name",
        regexp_replace(col("last_name"), "[^a-zA-Z]", "")
    )
)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 103, Finished, Available, Finished)

In [102]:
display(df)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 104, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d2529018-a8bf-4a6f-9fac-edb5848e87f4)

# Cleaning Products file

In [103]:
df_p = (
    spark.read.format("csv")
    .option("header", "true")
    .load(
        "abfss://cad6716d-54df-4655-aed7-d883bc9a65b4@onelake.dfs.fabric.microsoft.com/"
        "b90738e8-b697-45a2-a1e2-b96143f7b0f5/Files/Dataset/products.csv"
    )
)

display(df_p)


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 105, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c5ff40ef-1352-4af1-91b7-cde0b4d52c8f)

In [104]:
df_p.count()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 106, Finished, Available, Finished)

500

In [105]:
df_p=df_p.dropDuplicates()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 107, Finished, Available, Finished)

In [106]:
df_p.groupBy("product_id").count().filter("count > 1").show()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 108, Finished, Available, Finished)

+----------+-----+
|product_id|count|
+----------+-----+
+----------+-----+



In [107]:
# select() chooses columns- values inside a column, filter() chooses rows.
df_p.filter(col("product_id").isNull()).count()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 109, Finished, Available, Finished)

0

In [108]:
for c in df_p.columns:
    null_count= df_p.filter(col(c).isNull()).count()
    print(c, null_count)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 110, Finished, Available, Finished)

product_id 0
product_name 0
category 30
price 0


In [109]:
df_p.select("category").distinct().count()


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 111, Finished, Available, Finished)

47

In [110]:
df_p.filter(col("category").isNull()).count()


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 112, Finished, Available, Finished)

30

In [111]:
df_p=df_p.withColumn("product_name",lower(col("product_name")))

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 113, Finished, Available, Finished)

In [112]:
df_p=df_p.withColumn("price",regexp_replace(col("price"),"-",""))

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 114, Finished, Available, Finished)

In [113]:
df_p=df_p.withColumn("price",regexp_replace(col("price"),"(?i)^nan$","0"))

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 115, Finished, Available, Finished)

In [114]:
display(df_p)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 116, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f6fcfb0d-67e8-4661-be96-3830c7874235)

In [115]:
df_p=df_p.withColumn("category",lower(col("category")))

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 117, Finished, Available, Finished)

In [116]:
df_p.printSchema()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 118, Finished, Available, Finished)

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: string (nullable = true)



In [117]:
df_p = df_p.withColumn("price", col("price").cast("decimal(10,2)"))

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 119, Finished, Available, Finished)

In [118]:
df_p.printSchema()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 120, Finished, Available, Finished)

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: decimal(10,2) (nullable = true)



# Cleaning Tickets file

In [119]:
df_t = (
    spark.read.format("csv")
    .option("header", "true")
    .load(
        "abfss://cad6716d-54df-4655-aed7-d883bc9a65b4@onelake.dfs.fabric.microsoft.com/"
        "b90738e8-b697-45a2-a1e2-b96143f7b0f5/Files/Dataset/tickets.csv"
    )
)

display(df_t)


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 121, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 358d9e71-b321-411e-b0f2-9172d77e712b)

In [120]:
for c in df_t.columns:
    null_count= df_t.filter(col(c).isNull()).count()
    print(c, null_count)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 122, Finished, Available, Finished)

ticket_id 0
customer_id 0
issue_type 0
ticket_created 1789
ticket_resolved 1823
resolution_time_hours 0
sentiment 0
support_agent 0


In [121]:
df_t.count()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 123, Finished, Available, Finished)

30000

In [122]:
df_t.printSchema()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 124, Finished, Available, Finished)

root
 |-- ticket_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- issue_type: string (nullable = true)
 |-- ticket_created: string (nullable = true)
 |-- ticket_resolved: string (nullable = true)
 |-- resolution_time_hours: string (nullable = true)
 |-- sentiment: string (nullable = true)
 |-- support_agent: string (nullable = true)



In [123]:
df_t=df_t.dropDuplicates()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 125, Finished, Available, Finished)

In [124]:
df_t.count()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 126, Finished, Available, Finished)

30000

In [125]:
df_t = (
    df_t
    .withColumn("ticket_created", to_date(to_timestamp(col("ticket_created"))))
    .withColumn("ticket_resolved", to_date(to_timestamp(col("ticket_resolved"))))
)


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 127, Finished, Available, Finished)

In [126]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 128, Finished, Available, Finished)

In [127]:
display(df_t)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 129, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0e996a25-f525-4ddc-a6d1-93c07da45abc)

In [128]:
df_t.printSchema()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 130, Finished, Available, Finished)

root
 |-- ticket_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- issue_type: string (nullable = true)
 |-- ticket_created: date (nullable = true)
 |-- ticket_resolved: date (nullable = true)
 |-- resolution_time_hours: string (nullable = true)
 |-- sentiment: string (nullable = true)
 |-- support_agent: string (nullable = true)



In [129]:
df_t = df_t.withColumn(
    "resolution_time_hours",
    col("resolution_time_hours").cast("double").cast("int")
)


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 131, Finished, Available, Finished)

# Cleaning Events file

In [130]:
df_e = (
    spark.read.format("csv")
    .option("header", "true")
    .load(
        "abfss://cad6716d-54df-4655-aed7-d883bc9a65b4@onelake.dfs.fabric.microsoft.com/"
        "b90738e8-b697-45a2-a1e2-b96143f7b0f5/Files/Dataset/events.csv"
    )
)

display(df_e)


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 132, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 27618b9c-351a-4531-9aea-915011bbdc29)

In [131]:
df_e = (
    df_e
    .withColumn("timestamp", to_date(to_timestamp(col("timestamp"))))
)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 133, Finished, Available, Finished)

In [132]:
display(df_e)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 134, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b983cda2-4200-401a-904d-e3675b1f8566)

In [133]:
for c in df_e.columns:
    null_count= df_e.filter(col(c).isNull()).count()
    print(c, null_count)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 135, Finished, Available, Finished)

event_id 0
session_id 0
customer_id 150553
event_type 0
page_url 8305
device_id 0
timestamp 40171
ingest_run_id 0


In [134]:
df_e.printSchema()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 136, Finished, Available, Finished)

root
 |-- event_id: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- page_url: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- timestamp: date (nullable = true)
 |-- ingest_run_id: string (nullable = true)



# Cleaning Orders file

In [135]:
df_o = (
    spark.read.format("csv")
    .option("header", "true")
    .load(
        "abfss://cad6716d-54df-4655-aed7-d883bc9a65b4@onelake.dfs.fabric.microsoft.com/"
        "b90738e8-b697-45a2-a1e2-b96143f7b0f5/Files/Dataset/orders.csv"
    )
)

display(df_o)


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 137, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6e0c3ed3-7a73-4790-8d9a-c20728b3b937)

In [136]:
for c in df_o.columns:
    null_count= df_o.filter(col(c).isNull()).count()
    print(c, null_count)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 138, Finished, Available, Finished)

order_id 0
customer_id 0
product_id 0
order_amount 0
order_date 17914
payment_method 0
status 0
quantity 0


In [137]:
df_o=df_o.withColumn("order_amount",regexp_replace(col("order_amount"),"-",""))
df_o=df_o.withColumn("order_amount",regexp_replace(col("order_amount"),"(?i)^nan$","0"))

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 139, Finished, Available, Finished)

In [138]:
display(df_o)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 140, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d60f89a5-7c47-418d-9b30-9366b40bd768)

In [139]:
df_o = (
    df_o
    .withColumn("order_date", to_date(to_timestamp(col("order_date"))))
)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 141, Finished, Available, Finished)

In [140]:
df_o = (
    df_o
    .withColumn("order_amount", col("order_amount").cast("decimal(10,2)"))
    .withColumn("quantity", col("quantity").cast("int"))
)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 142, Finished, Available, Finished)

In [141]:
df_o.printSchema()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 143, Finished, Available, Finished)

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_amount: decimal(10,2) (nullable = true)
 |-- order_date: date (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- status: string (nullable = true)
 |-- quantity: integer (nullable = true)



In [142]:
display(df)
display(df_t)
display(df_e)
display(df_p)
display(df_o)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 144, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3e7aef71-5792-4961-8973-24e296741e2d)

SynapseWidget(Synapse.DataFrame, 3dd02699-ee22-4774-be0f-e3634329ecdd)

SynapseWidget(Synapse.DataFrame, c9270bb5-c7c7-4c62-86bf-6ef9c1a2091d)

SynapseWidget(Synapse.DataFrame, 6d61e493-bd8a-485e-b97b-fa339e810866)

SynapseWidget(Synapse.DataFrame, b74f4a90-e21c-4509-b0a3-c83d893d31ad)

In [143]:
df_t.select("issue_type").distinct().show(truncate=False)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 145, Finished, Available, Finished)

+----------+
|issue_type|
+----------+
|payment   |
| payment  |
|payment_  |
|refund    |
|del       |
|dnufer    |
| refund   |
|delay     |
|REFUND    |
|d3lay     |
|refundx   |
|yaled     |
| delay    |
|pro       |
|refund_   |
|refund    |
|DELAY     |
|product_  |
| product  |
|ref       |
+----------+
only showing top 20 rows



In [144]:
df_t.select("sentiment").distinct().show(truncate=False)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 146, Finished, Available, Finished)

+---------+
|sentiment|
+---------+
|positive |
|POSITIVE |
|NEGATIVE |
|negative |
|pos      |
|neg      |
|neutral  |
|negative |
| positive|
|neutral  |
|NEUTRAL  |
|positiv3 |
| neutral |
|neu      |
|n3utral  |
|positive |
| negative|
|n3gativ3 |
+---------+



In [145]:
df_p.select("category").distinct().show(50)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 147, Finished, Available, Finished)

+------------+
|    category|
+------------+
|     sports |
|         ele|
| 3l3ctronics|
|      b3auty|
|  automotive|
|     beauty-|
|        hom3|
|    kitchen |
| automotive |
|         hom|
|      beauty|
|       toys |
|         clo|
|     kitchen|
|     sports-|
|electronics |
|  automotiv3|
|     beauty |
|         kit|
|       home_|
|   clothing-|
|    kitchen-|
|   clothing |
|     kitch3n|
| electronics|
|       toys-|
|         bea|
|     beauty_|
|      sports|
|    clothing|
|   clothing_|
|        home|
|        toys|
|         aut|
|       home-|
|electronics_|
| automotive_|
|         toy|
|        NULL|
+------------+



In [146]:
df_o.select("quantity").distinct().show(50)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 148, Finished, Available, Finished)

+--------+
|quantity|
+--------+
|       1|
|       3|
|       5|
|       4|
|       2|
|      -3|
|       0|
|    NULL|
+--------+



In [147]:
df_o.select("status").distinct().show()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 149, Finished, Available, Finished)

+---------+
|   status|
+---------+
|refunded |
| refunded|
|      suc|
|  success|
|   failed|
|  Success|
|  SUCCESS|
|     fail|
|   FAILED|
|      REF|
+---------+



In [148]:
df_o.select("payment_method").distinct().show()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 150, Finished, Available, Finished)

+--------------+
|payment_method|
+--------------+
|        wallet|
|       wall-et|
|          cash|
|            cd|
|           upi|
|           upi|
|          CRAD|
|          card|
|        WALLET|
|          c@rd|
|          UPI |
+--------------+



In [149]:
df_t = df_t.withColumn(
    "issue_type",
    lower(trim(col("issue_type")))
)

# Remove spaces and underscores
df_t = df_t.withColumn(
    "issue_type",
    regexp_replace(col("issue_type"), r"[\s_]+", "")
)

# Standardize values
df_t = df_t.withColumn(
    "issue_type",
    when(col("issue_type").isin("payment", "paym3nt","paymentx","pay","tnemyap"), "payment")
    .when(col("issue_type").isin("refund", "refundx", "refund_", "ref", "refun","r3fund"), "refund")
    .when(col("issue_type").isin("delay", "del", "d3lay","delayx"), "delay")
    .when(col("issue_type").isin("product", "pro","productx","tcudorp"), "product")
    .when(col("issue_type") == "yaled", "delay")    # reversed delay
    .when(col("issue_type") == "dnufer", "refund")  # reversed refund
    .otherwise(col("issue_type"))   # keep remaining values as-is
)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 151, Finished, Available, Finished)

In [150]:
# Step 1: lowercase + trim
df_t = df_t.withColumn(
    "sentiment",
    lower(trim(col("sentiment")))
)

# Step 2: remove spaces
df_t = df_t.withColumn(
    "sentiment",
    regexp_replace(col("sentiment"), r"\s+", "")
)

# Step 3: standardize values
df_t = df_t.withColumn(
    "sentiment",
    when(col("sentiment").isin("positive", "pos", "positiv3"), "positive")
    .when(col("sentiment").isin("negative", "neg", "n3gativ3"), "negative")
    .when(col("sentiment").isin("neutral", "neu", "n3utral"), "neutral")
    .otherwise(col("sentiment"))
)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 152, Finished, Available, Finished)

In [151]:
from pyspark.sql.functions import col, lower, trim, regexp_replace, when

# 1) normalize: lowercase + trim
df_p = df_p.withColumn("category", lower(trim(col("category"))))

# 2) remove trailing/extra separators and spaces
df_p = df_p.withColumn("category", regexp_replace(col("category"), r"[\s\-_]+", ""))

# 3) fix common leetspeak: 3 -> e
df_p = df_p.withColumn("category", regexp_replace(col("category"), "3", "e"))

# 4) map dirty/truncated values to canonical categories
df_p = df_p.withColumn(
    "category",
    when(col("category").isNull() | (col("category") == "") | (col("category") == "null"), None)
    .when(col("category").isin("electronics", "ele"), "electronics")
    .when(col("category").isin("beauty", "bea", "beauty"), "beauty")
    .when(col("category").isin("home", "hom"), "home")
    .when(col("category").isin("kitchen", "kit", "kitchen"), "kitchen")
    .when(col("category").isin("clothing", "clo"), "clothing")
    .when(col("category").isin("sports"), "sports")
    .when(col("category").isin("toys", "toy"), "toys")
    .when(col("category").isin("automotive", "aut", "automotiv"), "automotive")
    .otherwise(col("category"))
)


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 153, Finished, Available, Finished)

In [152]:
# Convert word numbers
df_o = df_o.withColumn(
    "quantity",
    when(col("quantity") == "five", "5").otherwise(col("quantity"))
)

# Remove unwanted characters (keep digits, dot, minus)
df_o = df_o.withColumn(
    "quantity",
    regexp_replace(col("quantity"), "[^0-9.-]", "")
)

# 🔑 FIX: make negative values positive
df_o = df_o.withColumn(
    "quantity",
    abs(col("quantity")).cast("int")
)

df_o = df_o.withColumn(
    "quantity",
    when(col("quantity").isNull(), 0).otherwise(col("quantity"))
)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 154, Finished, Available, Finished)

In [153]:
# normalize text
df_o = df_o.withColumn("status", lower(trim(col("status"))))

# remove spaces
df_o = df_o.withColumn("status", regexp_replace(col("status"), r"\s+", ""))

# map to final categories
df_o = df_o.withColumn(
    "status",
    when(col("status").isin("success", "suc"), "success")
    .when(col("status").isin("failed", "fail"), "failed")
    .when(col("status").isin("refunded", "ref"), "refund")
    .otherwise(col("status"))
)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 155, Finished, Available, Finished)

In [154]:
# normalize text
df_o = df_o.withColumn("payment_method", lower(trim(col("payment_method"))))

# remove special characters
df_o = df_o.withColumn(
    "payment_method",
    regexp_replace(col("payment_method"), r"[^a-z]", "")
)

# map to final categories
df_o = df_o.withColumn(
    "payment_method",
    when(col("payment_method").isin("wallet"), "wallet")
    .when(col("payment_method").isin("cash"), "cash")
    .when(col("payment_method").isin("card", "crad", "crd","cd"), "card")
    .when(col("payment_method").isin("upi"), "upi")
    .otherwise(col("payment_method"))
)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 156, Finished, Available, Finished)

In [155]:
df_p = df_p.withColumn(
    "product_name",
    trim(regexp_replace(col("product_name"), "[^a-zA-Z ]", ""))
)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 157, Finished, Available, Finished)

In [156]:
display(df)
display(df_t)
display(df_e)
display(df_p)
display(df_o)

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 158, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3b1e1a80-03bb-46be-85aa-ff869267690b)

SynapseWidget(Synapse.DataFrame, 25054da7-4313-481e-a8d5-2fb4bab09ffb)

SynapseWidget(Synapse.DataFrame, 4d86b23c-af05-4e8b-93ca-c425f73c0b08)

SynapseWidget(Synapse.DataFrame, 49a3d7a2-b0c4-4068-940b-23ac8195e289)

SynapseWidget(Synapse.DataFrame, 4d95f157-f457-442a-b692-1f8d92c22c97)

In [157]:
df_o.count()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 159, Finished, Available, Finished)

300000

In [158]:
df_t.count()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 160, Finished, Available, Finished)

30000

In [159]:
df_e.count()

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 161, Finished, Available, Finished)

500000

In [160]:
df = df.withColumnRenamed("device_id(s)", "device_ids")

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 162, Finished, Available, Finished)

In [161]:
df_t = df_t.withColumnRenamed("device_id(s)", "device_ids")

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 163, Finished, Available, Finished)

In [162]:
df_o = df_o.withColumnRenamed("device_id(s)", "device_ids")


StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 164, Finished, Available, Finished)

In [163]:
df.write.mode("overwrite").format("delta").saveAsTable("silver_customers")
df_o.write.mode("overwrite").format("delta").saveAsTable("silver_orders")
df_p.write.mode("overwrite").format("delta").saveAsTable("silver_products")
df_e.write.mode("overwrite").format("delta").saveAsTable("silver_events")
df_t.write.mode("overwrite").format("delta").saveAsTable("silver_tickets")

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 165, Finished, Available, Finished)

In [164]:
display(df.limit(5))
display(df_t.limit(5))
display(df_e.limit(5))
display(df_p.limit(5))
display(df_o.limit(5))

StatementMeta(, d8d5653d-9250-4bb2-9dd8-dd6d0d28f766, 166, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 95aaf189-fc4c-4f00-b146-fd0105416c57)

SynapseWidget(Synapse.DataFrame, 2f9d940e-ff17-4aa4-a22f-d85ac0599026)

SynapseWidget(Synapse.DataFrame, 937f7335-b6a9-4ecc-a36d-20064e5386f8)

SynapseWidget(Synapse.DataFrame, 02e6a3c8-d4a0-418c-9f33-cf30566873ac)

SynapseWidget(Synapse.DataFrame, ae2b44fb-7708-4b96-994b-2628d3d89721)